###Config

In [ ]:
from sqlalchemy import create_engine
import sqlite3
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import json
# Ruta al archivo
file_path = "/content/drive/MyDrive/Facultad/PFC/Implementacion/config.json"

# Leer el archivo
with open(file_path, 'r') as file:
    config = json.load(file)

###API KEY

In [ ]:
import os

# os.environ["SEC_API_USER_AGENT"] =

os.environ["BEA_API_KEY"] = config["BEA_API_KEY"]

# os.environ["FRED_API_KEY"] =

###Install


In [ ]:
!pip install finagg
!finagg install --help

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.8/149.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.30
    Uninstalling yfinance-0.2.30:
      Successfully uninstalled yfinance-0.2.30
Usage: finagg install [OPTIONS]

  Set API keys/user agents, drop and recreate
  tables, and install the recommended datasets
  into the SQL database.

Options:
  -s, --skip [bea|fred|indices|sec|yfinance|fundam]
                                  Subpackage
                                  installations to
                                  skip. Useful to
                                  avoid
                                  re

###Inspect Tables

In [ ]:
from sqlalchemy import create_engine, inspect, MetaData

db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
# Combine all metadata
all_metadata = MetaData()
# Setup the database and create tables based on the combined metadata
engine = create_engine(f'sqlite:///{db_path}')
all_metadata.create_all(engine)

# Reflect the tables from the engine into the metadata
all_metadata.reflect(bind=engine)

# Get a list of all the tables
tables = all_metadata.tables.keys()

# Use an inspector to fetch columns and other details
inspector = inspect(engine)

#Ver datos
for table in tables:
    print("Table:", table)
    for column in inspector.get_columns(table):
        print("  Column:", column['name'], column['type'])

Streaming output truncated to the last 5000 lines.
  Column: year INTEGER
  Column: quarter INTEGER
  Column: value FLOAT
  Column: norm FLOAT
Table: gdp_22_524
  Column: year INTEGER
  Column: quarter INTEGER
  Column: value FLOAT
Table: gdp_22_524_z_norm
  Column: year INTEGER
  Column: quarter INTEGER
  Column: value FLOAT
  Column: norm FLOAT
Table: gdp_22_525
  Column: year INTEGER
  Column: quarter INTEGER
  Column: value FLOAT
Table: gdp_22_525_z_norm
  Column: year INTEGER
  Column: quarter INTEGER
  Column: value FLOAT
  Column: norm FLOAT
Table: gdp_22_52_z_norm
  Column: year INTEGER
  Column: quarter INTEGER
  Column: value FLOAT
  Column: norm FLOAT
Table: gdp_22_53
  Column: year INTEGER
  Column: quarter INTEGER
  Column: value FLOAT
Table: gdp_22_531
  Column: year INTEGER
  Column: quarter INTEGER
  Column: value FLOAT
Table: gdp_22_531_z_norm
  Column: year INTEGER
  Column: quarter INTEGER
  Column: value FLOAT
  Column: norm FLOAT
Table: gdp_22_532RL
  Column: year 

###Number of Rows

In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    table_name = table[0]
    # Colocar el nombre de la tabla entre comillas dobles
    cursor.execute(f'SELECT COUNT(*) FROM "{table_name}";')
    count = cursor.fetchone()[0]
    print(f"Table {table_name} has {count} rows.")

Streaming output truncated to the last 5000 lines.
Table fix_as_FAAt301E_k1n48401eq00_39 has 75 rows.
Table fix_as_FAAt301E_k1n48501eq00_40 has 75 rows.
Table fix_as_FAAt301E_k1n48601eq00_41 has 75 rows.
Table fix_as_FAAt301E_k1n487S1eq00_42 has 75 rows.
Table fix_as_FAAt301E_k1n49301eq00_43 has 75 rows.
Table fix_as_FAAt301E_k1n51001eq00_44 has 75 rows.
Table fix_as_FAAt301E_k1n51101eq00_45 has 75 rows.
Table fix_as_FAAt301E_k1n51201eq00_46 has 75 rows.
Table fix_as_FAAt301E_k1n51301eq00_47 has 75 rows.
Table fix_as_FAAt301E_k1n51401eq00_48 has 75 rows.
Table fix_as_FAAt301E_k1n52001eq00_49 has 75 rows.
Table fix_as_FAAt301E_k1n52101eq00_50 has 75 rows.
Table fix_as_FAAt301E_k1n52201eq00_51 has 75 rows.
Table fix_as_FAAt301E_k1n52301eq00_52 has 75 rows.
Table fix_as_FAAt301E_k1n52401eq00_53 has 75 rows.
Table fix_as_FAAt301E_k1n52501eq00_54 has 75 rows.
Table fix_as_FAAt301E_k1n53201eq00_57 has 75 rows.
Table fix_as_FAAt301E_k1n54001eq00_58 has 75 rows.
Table fix_as_FAAt301E_k1n54111e

###Tickers

In [ ]:
db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT DISTINCT ticker FROM 'fundam.refined.fundam'")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

('A',)
('AAPL',)
('ALGN',)
('ALLE',)
('AMAT',)
('AMCR',)
('ANET',)
('APTV',)
('ATVI',)
('AVGO',)
('AXON',)
('CARR',)
('CDW',)
('CLX',)
('COO',)
('COST',)
('CPRT',)
('CRL',)
('CSCO',)
('CTLT',)
('CVS',)
('CZR',)
('D',)
('DD',)
('DVA',)
('DXCM',)
('EA',)
('ECL',)
('EIX',)
('ENPH',)
('EW',)
('FANG',)
('FE',)
('FSLR',)
('FTNT',)
('GE',)
('GEHC',)
('GM',)
('GNRC',)
('GOOG',)
('GPN',)
('HCA',)
('HD',)
('HII',)
('HLT',)
('HPE',)
('HSIC',)
('HWM',)
('IDXX',)
('INCY',)
('ISRG',)
('JNPR',)
('KDP',)
('KMI',)
('KVUE',)
('LCID',)
('LDOS',)
('LH',)
('LHX',)
('LOW',)
('LRCX',)
('LULU',)
('LVS',)
('LW',)
('MAS',)
('MDT',)
('MELI',)
('MGM',)
('MHK',)
('MLM',)
('MMM',)
('MPC',)
('MPWR',)
('MRNA',)
('MRO',)
('MSFT',)
('MTD',)
('MU',)
('NCLH',)
('NEE',)
('NOC',)
('NTAP',)
('ON',)
('OTIS',)
('PAYC',)
('PNR',)
('PODD',)
('POOL',)
('PPG',)
('PWR',)
('QCOM',)
('QRVO',)
('REGN',)
('RMD',)
('ROL',)
('RVTY',)
('SBUX',)
('SEE',)
('SIRI',)
('SJM',)
('SLB',)
('SNPS',)
('SPGI',)
('STE',)
('STLD',)
('STX',)
('SWKS',)

###Data Range

In [ ]:
db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
# Combine all metadata
# Setup the database and create tables based on the combined metadata
engine = create_engine(f'sqlite:///{db_path}')

def get_date_range_from_table(engine, table_name, column_name, value):
    with engine.connect() as connection:
        query = text(f"""
            SELECT MIN(date), MAX(date)
            FROM "{table_name}"
            WHERE {column_name} = :value
        """)
        result = connection.execute(query, {"value": value}).fetchone()
        return result if result else (None, None)


def get_SEC_date_range_from_table(engine, table_name, column_name, value):
    with engine.connect() as connection:
        query = text(f"""
            SELECT MIN(fy), MAX(fy), MIN(fp), MAX(fp)
            FROM "{table_name}"
            WHERE {column_name} = :value
        """)
        result = connection.execute(query, {"value": value}).fetchone()
        if result:
            min_year, max_year, min_period, max_period = result
            start_date = f"{min_year}-{min_period}"
            end_date = f"{max_year}-{max_period}"
            return start_date, end_date
        else:
            return None, None

def get_fred_range(engine):
    with engine.connect() as connection:
        query = text(f"""
            SELECT MIN(date), MAX(date)
            FROM "fred.refined.economic"
        """)
        result = connection.execute(query).fetchone()
        return result if result else (None, None)


def get_bea_range(engine,table_name):
    with engine.connect() as connection:
        query = text(f"""
            SELECT MIN(year), MAX(year)
            FROM "{table_name}"
        """)
        result = connection.execute(query).fetchone()
        return result if result else (None, None)

In [ ]:
from sqlalchemy import text

ticker = "AAPL"  # Coloca el ticker que quieres consultar aquí
ticker_tables = ["yfinance.refined.daily", "fundam.refined.fundam", "fundam.refined.fundam.normalized"]  # tablas

cik_query = text("""
  SELECT cik
  FROM "sec.raw.submissions"
  WHERE ticker = :ticker
  LIMIT 1
""")
with engine.connect() as connection:
    cik = connection.execute(cik_query, {'ticker': ticker}).scalar()

cik_tables = ["sec.refined.quarterly.normalized"]  # Agrega las demás tablas que correspondan a 'cik'

#Macroeconomic data
start_date, end_date = get_fred_range(engine)
print(f"Table fred.refined.economic: Data range: {start_date} a {end_date}")

print("Datos BEA:")
start_date, end_date = get_bea_range(engine,"fixed_assets")
print(f"Table fixed_assets: Data range: {start_date} to {end_date}")

start_date, end_date = get_bea_range(engine,"gdp_by_industry")
print(f"Table gdp_by_industry: Data range: {start_date} to {end_date}")

start_date, end_date = get_bea_range(engine,"input_output")
print(f"Table input_output: Data range: {start_date} to {end_date}")


# Obtener rangos de fechas para tablas basadas en ticker
print(f"Empresa: {ticker}")
for table in ticker_tables:
  start_date, end_date = get_date_range_from_table(engine, table, "ticker", ticker)
  print(f"Table {table}: Data range: {start_date} to {end_date}")

# Obtener rangos de fechas para tablas basadas en cik
start_date, end_date = get_SEC_date_range_from_table(engine, "sec.refined.quarterly.normalized", "cik", cik)
print(f"Table sec.refined.quarterly.normalized: Data range: {start_date} to {end_date}")


Table fred.refined.economic: Data range: 2014-10-02 a 2023-08-28
Datos BEA:
Table fixed_assets: Data range: 1901 to 2021
Table gdp_by_industry: Data range: 2005 to 2023
Table input_output: Data range: 1997 to 2021
Empresa: AAPL
Table yfinance.refined.daily: Data range: 1980-12-15 to 2023-08-29
Table fundam.refined.fundam: Data range: 2010-01-25 to 2023-08-29
Table fundam.refined.fundam.normalized: Data range: 2010-01-25 to 2023-08-29
Table sec.refined.quarterly.normalized: Data range: 2010-Q1 to 2023-Q3


###Data SEC for Specific Ticker

In [ ]:
# def get_data_sec_ticker(engine, ticker):
#     cik_query = text("""
#       SELECT cik
#       FROM "sec.raw.submissions"
#       WHERE ticker = :ticker
#       LIMIT 1
#     """)
#     with engine.connect() as connection:
#         value_cik = connection.execute(cik_query, {'ticker': ticker}).scalar()

#     with engine.connect() as connection:
#         query = text(f"""
#             SELECT *
#             FROM "sec.refined.quarterly.normalized"
#             WHERE cik = :value
#         """)
#         result = connection.execute(query, {"value": value_cik}).fetchall()
#         return result if result else None

# ticker = "AAPL"
# data = get_data_sec_ticker(engine,ticker)
# for d in data:
#   print(d)

In [ ]:
import sqlite3
import pandas as pd

def get_data_sec_ticker(db_path, ticker):
    connection = sqlite3.connect(db_path)
    cik_query = """
      SELECT cik
      FROM "sec.raw.submissions"
      WHERE ticker = ?
      LIMIT 1
    """
    value_cik = connection.execute(cik_query, (ticker,)).fetchone()[0]

    query = f"""
        SELECT *
        FROM "sec.raw.tags"
        WHERE cik = ?
    """
    data = pd.read_sql(query, connection, params=(value_cik,))
    connection.close()

    return data

ticker = "AAPL"
data = get_data_sec_ticker(db_path, ticker)
data


,cik,accn,taxonomy,tag,form,units,fy,fp,start,end,filed,frame,label,description,entity,value
0,0000320193,0001193125-09-214859,us-gaap,Assets,10-K,USD,2009,FY,None,2008-09-27,2009-10-27,None,Assets,Sum of the carrying amounts as of the balance ...,Apple Inc.,3.957200e+10
1,0000320193,0001193125-09-153165,us-gaap,Assets,10-Q,USD,2009,Q3,None,2008-09-27,2009-07-22,CY2009Q2I,Assets,Sum of the carrying amounts as of the balance ...,Apple Inc.,3.957200e+10
2,0000320193,0001193125-10-238044,us-gaap,Assets,10-K,USD,2010,FY,None,2008-09-27,2010-10-27,CY2008Q3I,Assets,Sum of the carrying amounts as of the balance ...,Apple Inc.,3.617100e+10
3,0000320193,0001193125-10-012085,us-gaap,Assets,10-Q,USD,2010,Q1,None,2009-09-26,2010-01-25,CY2009Q4I,Assets,Sum of the carrying amounts as of the balance ...,Apple Inc.,4.750100e+10
4,0000320193,0001193125-10-088957,us-gaap,Assets,10-Q,USD,2010,Q2,None,2009-09-26,2010-04-21,CY2010Q1I,Assets,Sum of the carrying amounts as of the balance ...,Apple Inc.,4.750100e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,0000320193,0000320193-22-000059,us-gaap,StockholdersEquity,10-Q,USD,2022,Q2,None,2020-09-26,2022-04-29,CY2020Q4I,Stockholders' Equity Attributable to Parent,Total of all stockholders' equity (deficit) it...,Apple Inc.,6.533900e+10
509,0000320193,0000320193-22-000070,us-gaap,StockholdersEquity,10-Q,USD,2022,Q3,None,2020-09-26,2022-07-29,CY2021Q1I,Stockholders' Equity Attributable to Parent,Total of all stockholders' equity (deficit) it...,Apple Inc.,6.533900e+10
510,0000320193,0000320193-23-000006,us-gaap,StockholdersEquity,10-Q,USD,2023,Q1,None,2021-09-25,2023-02-03,None,Stockholders' Equity Attributable to Parent,Total of all stockholders' equity (deficit) it...,Apple Inc.,6.309000e+10
511,0000320193,0000320193-23-000064,us-gaap,StockholdersEquity,10-Q,USD,2023,Q2,None,2021-09-25,2023-05-05,CY2021Q4I,Stockholders' Equity Attributable to Parent,Total of all stockholders' equity (deficit) it...,Apple Inc.,6.309000e+10


###Show Prices

In [ ]:
def get_data_price(db_path, ticker):
    connection = sqlite3.connect(db_path)
    query = f"""
        SELECT *
        FROM "yfinance.refined.daily"
        WHERE ticker = ?
    """
    data = pd.read_sql(query, connection, params=(ticker,))
    connection.close()

    return data

ticker = "AAPL"
data = get_data_price(db_path, ticker)
data

,ticker,date,LOG_CHANGE(open),LOG_CHANGE(high),LOG_CHANGE(low),LOG_CHANGE(close),LOG_CHANGE(volume),"LOG_CHANGE(high, open)","LOG_CHANGE(low, open)"
0,AAPL,1980-12-15,-0.049004,-0.053342,-0.053581,-0.053581,-0.980845,0.000000,-0.004576
1,AAPL,1980-12-16,-0.075869,-0.075869,-0.076231,-0.076231,-0.508959,0.000000,-0.004938
2,AAPL,1980-12-17,0.019511,0.024330,0.024449,0.024449,-0.201401,0.004819,0.000000
3,AAPL,1980-12-18,0.028580,0.028445,0.028580,0.028580,-0.162870,0.004684,0.000000
4,AAPL,1980-12-19,0.059239,0.058970,0.059239,0.059239,-0.412351,0.004415,0.000000
...,...,...,...,...,...,...,...,...,...
10762,AAPL,2023-08-23,0.008212,0.021547,0.011732,0.021711,0.225376,0.016830,-0.001065
10763,AAPL,2023-08-24,0.011971,-0.002482,-0.013095,-0.026519,0.041299,0.002377,-0.026131
10764,AAPL,2023-08-25,-0.018378,-0.010826,-0.001080,0.012564,-0.066345,0.009929,-0.008834
10765,AAPL,2023-08-28,0.015162,0.008006,0.015408,0.008807,-0.161060,0.002773,-0.008588


###Show Tags Type

In [ ]:
import sqlite3
import pandas as pd

def get_tags(db_path):
    connection = sqlite3.connect(db_path)
    query = f"""
        SELECT DISTINCT tag
        FROM "sec.raw.tags"
    """
    data = pd.read_sql(query, connection)
    #data = pd.read_sql(query, connection)

    connection.close()

    return data

data = get_tags(db_path)
data

,tag
0,Assets
1,AssetsCurrent
2,EarningsPerShareBasic
3,InventoryNet
4,LiabilitiesCurrent
5,NetIncomeLoss
6,StockholdersEquity
7,CommonStockSharesOutstanding
8,Liabilities


###Show Tags by Ticker

In [ ]:
def get_tags_ticker(db_path, ticker):
    #Get the company cik by ticker
    connection = sqlite3.connect(db_path)
    cik_query = """
      SELECT cik
      FROM "sec.raw.submissions"
      WHERE ticker = ?
      LIMIT 1
    """
    value_cik = connection.execute(cik_query, (ticker,)).fetchone()[0]

    query = f"""
        SELECT *
        FROM "sec.refined.quarterly"
        WHERE cik = ?
    """
    data = pd.read_sql(query, connection, params=(value_cik,))
    #data = pd.read_sql(query, connection)

    connection.close()

    return data

ticker = "MSFT"
data = get_tags_ticker(db_path, ticker)
data

,cik,filed,fy,fp,LOG_CHANGE(Assets),LOG_CHANGE(AssetsCurrent),LOG_CHANGE(CommonStockSharesOutstanding),LOG_CHANGE(InventoryNet),LOG_CHANGE(Liabilities),LOG_CHANGE(LiabilitiesCurrent),LOG_CHANGE(StockholdersEquity),AssetCoverageRatio,BookRatio,DebtEquityRatio,EarningsPerShareBasic,QuickRatio,ReturnOnAssets,ReturnOnEquity,WorkingCapitalRatio
0,0000789019,2011-01-27,2011,Q2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071871,1.501477,5.327065,0.901881,1.15,2.101044,0.118867,0.231150,2.129346
1,0000789019,2011-04-28,2011,Q3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031716,1.501477,5.327065,0.873726,1.61,2.101044,0.165387,0.311573,2.129346
2,0000789019,2011-10-20,2012,Q1,0.232968,0.296845,-0.034268,0.617375,0.256600,0.095738,0.026596,1.548401,6.815067,1.099676,0.63,2.555988,0.049768,0.115249,2.603670
3,0000789019,2012-04-19,2012,Q3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.129871,1.548401,6.815067,0.965745,2.03,2.555988,0.158927,0.323206,2.603670
4,0000789019,2012-10-18,2013,Q1,0.109399,0.127245,0.000597,-0.187876,0.061730,0.127536,0.105359,1.613299,7.918268,0.924517,0.68,2.568129,0.047316,0.096614,2.602912
5,0000789019,2013-01-24,2013,Q2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076629,1.613299,7.918268,0.856319,1.47,2.568129,0.101937,0.192792,2.602912
6,0000789019,2013-04-18,2013,Q3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.068380,1.613299,7.918268,0.799720,2.08,2.568129,0.144058,0.254446,2.602912
7,0000789019,2013-10-24,2014,Q1,0.160830,0.176085,-0.006344,0.533263,0.145176,0.135117,0.002575,1.654102,9.479347,0.922294,0.53,2.659967,0.031356,0.064879,2.711762
8,0000789019,2014-01-23,2014,Q2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052907,1.654102,9.479347,0.874766,1.29,2.659967,0.076128,0.149402,2.711762
9,0000789019,2014-04-24,2014,Q3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055111,1.654102,9.479347,0.827861,2.02,2.659967,0.118640,0.220347,2.711762


###BEA Data

In [ ]:
import finagg
import pandas as pd
from sqlalchemy import text

def show_bea_data(engine,table):
  with engine.begin() as conn:
      query = text(f"""SELECT * FROM {table}""")
      df = pd.read_sql_query(query, conn)
  return df

####Input_Output

In [ ]:
db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
engine = create_engine(f'sqlite:///{db_path}')

with engine.connect() as connection:
    result = connection.execute(text("SELECT sql FROM sqlite_master WHERE name = 'input_output';")) #Datos de la table
    for row in result:
        print(row)

('CREATE TABLE input_output (\n\ttable_id INTEGER NOT NULL, \n\tyear INTEGER NOT NULL, \n\trow_code VARCHAR NOT NULL, \n\trow_description VARCHAR, \n\t ... (23 characters truncated) ... ol_code VARCHAR NOT NULL, \n\tcol_description VARCHAR, \n\tcol_type VARCHAR, \n\tvalue FLOAT, \n\tPRIMARY KEY (table_id, year, row_code, col_code)\n)',)


In [ ]:
df_in_out=show_bea_data(engine,"input_output")
df_in_out

,table_id,year,row_code,row_description,row_type,col_code,col_description,col_type,value
0,6000,1997,,Total commodity output,Commodity,311FT,Food and beverage and tobacco products,Commodity,2.673929e+00
1,6000,1997,,Total commodity output,Commodity,481,Air transportation,Commodity,1.951930e+00
2,6000,1997,,Total commodity output,Commodity,487OS,Other transportation and support activities,Commodity,1.653248e+00
3,6000,1997,,Total commodity output,Commodity,22,Utilities,Commodity,1.700775e+00
4,6000,1997,,Total commodity output,Commodity,493,Warehousing and storage,Commodity,1.354362e+00
...,...,...,...,...,...,...,...,...,...
587341,6010,2021,VAPRO,Value Added (producer prices),Commodity,22,Utilities,Industry,3.784360e+05
587342,6010,2021,VAPRO,Value Added (producer prices),Commodity,81,"Other services, except government",Industry,4.698510e+05
587343,6010,2021,VAPRO,Value Added (producer prices),Commodity,21,Mining,Industry,3.339350e+05
587344,6010,2021,VAPRO,Value Added (producer prices),Commodity,11,"Agriculture, forestry, fishing, and hunting",Industry,2.065680e+05


######Tipos de tablas con su descripcion

In [ ]:
with engine.connect() as connection:
    result = connection.execute(text("SELECT DISTINCT row_code FROM input_output")) #Datos de la table
    for row in result:
        print(row)


('',)
('111CA',)
('113FF',)
('211',)
('212',)
('213',)
('22',)
('23',)
('311FT',)
('313TT',)
('315AL',)
('321',)
('322',)
('323',)
('324',)
('325',)
('326',)
('327',)
('331',)
('332',)
('333',)
('334',)
('335',)
('3361MV',)
('3364OT',)
('337',)
('339',)
('42',)
('441',)
('445',)
('452',)
('481',)
('482',)
('483',)
('484',)
('485',)
('486',)
('487OS',)
('493',)
('4A0',)
('511',)
('512',)
('513',)
('514',)
('521CI',)
('523',)
('524',)
('525',)
('532RL',)
('5411',)
('5412OP',)
('5415',)
('55',)
('561',)
('562',)
('61',)
('621',)
('622',)
('623',)
('624',)
('711AS',)
('713',)
('721',)
('722',)
('81',)
('GFE',)
('GFGD',)
('GFGN',)
('GSLE',)
('GSLG',)
('HS',)
('ORE',)
('Other',)
('Used',)
('11',)
('21',)
('31G',)
('44RT',)
('48TW',)
('51',)
('6',)
('7',)
('FIRE',)
('G',)
('PROF',)
('T017',)
('T005',)
('T00OTOP',)
('T00SUB',)
('T00TOP',)
('T018',)
('V001',)
('V003',)
('VABAS',)
('VAPRO',)
('T00OSUB',)


In [ ]:
table_id = 6000
while(table_id!=6011):
  with engine.connect() as connection:
      result = connection.execute(text(f"SELECT * FROM input_output WHERE table_id={table_id} LIMIT 5")) #Datos de la table
      for row in result:
          print(row)
      table_id+=1

(6000, 1997, '', 'Total commodity output', 'Commodity', '111CA', 'Farms', 'Commodity', 2.270805597305298)
(6000, 1997, '', 'Total commodity output', 'Commodity', '113FF', 'Forestry, fishing, and related activities', 'Commodity', 1.939224123954773)
(6000, 1997, '', 'Total commodity output', 'Commodity', '211', 'Oil and gas extraction', 'Commodity', 1.8392596244812012)
(6000, 1997, '', 'Total commodity output', 'Commodity', '212', 'Mining, except oil and gas', 'Commodity', 2.108003616333008)
(6000, 1997, '', 'Total commodity output', 'Commodity', '213', 'Support activities for mining', 'Commodity', 1.679688811302185)
(6001, 1997, '', 'Total commodity output', 'Commodity', '11', 'Agriculture, forestry, fishing, and hunting', 'Commodity', 2.1731319427490234)
(6001, 1997, '', 'Total commodity output', 'Commodity', '21', 'Mining', 'Commodity', 1.9003849029541016)
(6001, 1997, '', 'Total commodity output', 'Commodity', '22', 'Utilities', 'Commodity', 1.6299370527267456)
(6001, 1997, '', 'Tota

In [ ]:
finagg.bea.api.input_output.get_parameter_values("TableID")  #El key no se condicen con los table_id. Voy a asumir que van en orden.

,Key,Desc
0,262,The Domestic Supply of Commodities by Industri...
1,261,The Domestic Supply of Commodities by Industri...
2,259,The Use of Commodities by Industries - Summary
3,258,The Use of Commodities by Industries - Sector


####Fixed_Assets

In [29]:
df=show_bea_data(engine,"fixed_assets")
df


,table_id,series_code,line,line_description,year,metric,units,e,value
0,FAAt101,k1wtotl1es00,1,Fixed assets and consumer durable goods,1925,Current Dollars,Level,9,297.500000
1,FAAt101,k1wtotl1es00,1,Fixed assets and consumer durable goods,1926,Current Dollars,Level,9,307.600006
2,FAAt101,k1wtotl1es00,1,Fixed assets and consumer durable goods,1927,Current Dollars,Level,9,315.500000
3,FAAt101,k1wtotl1es00,1,Fixed assets and consumer durable goods,1928,Current Dollars,Level,9,328.200012
4,FAAt101,k1wtotl1es00,1,Fixed assets and consumer durable goods,1929,Current Dollars,Level,9,333.600006
...,...,...,...,...,...,...,...,...,...
542684,FAAt903,ixgstlc1es00,24,State and local,2017,Chained Dollars,Level,9,358.700012
542685,FAAt903,ixgstlc1es00,24,State and local,2018,Chained Dollars,Level,9,367.200012
542686,FAAt903,ixgstlc1es00,24,State and local,2019,Chained Dollars,Level,9,372.799988
542687,FAAt903,ixgstlc1es00,24,State and local,2020,Chained Dollars,Level,9,375.500000


In [41]:
grouped = df.groupby(['table_id', 'series_code'])

for (table_id, series_code), group_df in grouped:
    unique_lines = group_df['line'].unique()

    if len(unique_lines) > 1:
        print(f"For table_id={table_id} and series_code={series_code}:")
        print("Unique lines:", unique_lines)
        print("Unique line descriptions:", group_df['line_description'].unique())
        print("---------------------------------------------------")


For table_id=FAAt101 and series_code=k1gtotl1es00:
Unique lines: [ 9 22]
Unique line descriptions: ['Government' 'Government fixed assets']
---------------------------------------------------
For table_id=FAAt101 and series_code=k1ttotl1es00:
Unique lines: [ 2 16]
Unique line descriptions: ['Fixed assets' 'Private and government fixed assets']
---------------------------------------------------
For table_id=FAAt102 and series_code=kcgtotl1es00:
Unique lines: [ 9 22]
Unique line descriptions: ['Government' 'Government fixed assets']
---------------------------------------------------
For table_id=FAAt102 and series_code=kcttotl1es00:
Unique lines: [ 2 16]
Unique line descriptions: ['Fixed assets' 'Private and government fixed assets']
---------------------------------------------------
For table_id=FAAt103 and series_code=m1gtotl1es00:
Unique lines: [ 9 22]
Unique line descriptions: ['Government' 'Government fixed assets']
---------------------------------------------------
For table_id

In [37]:
df["series_code"].unique()

array(['k1wtotl1es00', 'k1ttotl1es00', 'k1ptotl1es00', ...,
       'ixx53101es00', 'ixgfedr1es00', 'ixgstlc1es00'], dtype=object)

In [35]:
table_id_filter = "FAAt903"
grouped_descriptions = df[df['table_id'] == table_id_filter].groupby(['series_code'])['line_description'].unique()
print(grouped_descriptions)



series_code
ixctotl1cd00                             [Consumer durable goods]
ixgfedr1es00                                            [Federal]
ixgstlc1es00                                    [State and local]
ixgtotl1eq00                                          [Equipment]
ixgtotl1es00                [Government, Government fixed assets]
ixgtotl1esnr                                     [Nonresidential]
ixgtotl1ip00                     [Intellectual property products]
ixgtotl1sa00                                        [Residential]
ixgtotl1stnr                                         [Structures]
ixntotl1eq00                                          [Equipment]
ixntotl1es00                                     [Nonresidential]
ixntotl1ip00                     [Intellectual property products]
ixntotl1st00                                         [Structures]
ixptotl1es00                                            [Private]
ixr53101es00                                        [Residential

####Eliminar tablas fix_as

In [ ]:
# from sqlalchemy import create_engine, MetaData

# db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
# engine = create_engine(f'sqlite:///{db_path}')

# # Crear un objeto MetaData
# metadata = MetaData()

# # Conectar al motor y recuperar la lista de todas las tablas
# metadata.reflect(engine)

# # Filtrar las tablas que comienzan con "fix_as_"
# tables_to_drop = [table for name, table in metadata.tables.items() if name.startswith('fix_as_')]

# # Eliminar cada tabla filtrada
# for table in tables_to_drop:
#     table.drop(engine)

# print("Tablas eliminadas con éxito!")


Tablas eliminadas con éxito!


####Crear tablas

In [ ]:
# from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, Index, PrimaryKeyConstraint
# import finagg

# db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
# engine = create_engine(f'sqlite:///{db_path}')

# # Ordenar y agrupar el DataFrame
# df=show_bea_data(engine,"fixed_assets")
# grouped = df.groupby(['table_id', 'series_code', 'line'])


# metadata = MetaData()

# # Iterar sobre el objeto 'grouped'
# for (table_id, series_code,line), group_df in grouped:
#     table_name = f"fix_as_{table_id}_{series_code}_{line}"


#     # duplicates = group_df[group_df.duplicated(subset='year', keep=False)]
#     # if not duplicates.empty:
#     #   print(f"Duplicados encontrados para {table_id}, {series_code}:")
#     #   print(duplicates)

#     # Excluir las columnas que no queremos
#     group_df = group_df.drop(columns=['table_id', 'series_code', 'units', 'line'])

#     group_df["log_change"] = finagg.utils.safe_log_change(group_df["value"])

#     # Normalización (z-score) de los valores originales
#     group_df['norm'] = (group_df['value'] - group_df['value'].mean()) / group_df['value'].std()

#     # Normalización (z-score) de los logaritmos
#     group_df['norm_log_change'] = (group_df['log_change'] - group_df['log_change'].mean()) / group_df['log_change'].std()


#     # Definir la estructura de la tabla
#     table = Table(table_name, metadata,
#                   Column('year', Integer),
#                   Column('line_description', String),
#                   Column('metric', String),
#                   Column('e', Float),
#                   Column('value', Float),
#                   Column('log_change', Float),
#                   Column('norm', Float),
#                   Column('norm_log_change', Float),
#                   PrimaryKeyConstraint('year')
#                   )

#     # Crear la tabla en la base de datos
#     table.create(engine)

#     # Insertar datos en la tabla recién creada
#     group_df.to_sql(table_name, engine, if_exists='append', index=False)

####Contar tablas

In [43]:
from sqlalchemy import create_engine, MetaData

db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
engine = create_engine(f'sqlite:///{db_path}')

# Crear un objeto MetaData
metadata = MetaData()

# Conectar al motor y recuperar la lista de todas las tablas
metadata.reflect(engine)

# Filtrar las tablas que comienzan con "fix_as_"
fix_as_tables = [name for name, table in metadata.tables.items() if name.startswith('fix_as_')]

# Contar y mostrar el número de tablas
num_tables = len(fix_as_tables)
print(f"El número de tablas que comienzan con 'fix_as_' es: {num_tables}")



El número de tablas que comienzan con 'fix_as_' es: 6261


In [ ]:
df_b = show_bea_data(engine,"fix_as_FAAt101_k1gtotl1es00_22")
df_b

,year,line_description,metric,e,value,log_change,norm,norm_log_change
0,1925,Government fixed assets,Current Dollars,9.0,38.299999,NaN,-0.764517,NaN
1,1926,Government fixed assets,Current Dollars,9.0,38.900002,0.015544,-0.764393,-0.771567
2,1927,Government fixed assets,Current Dollars,9.0,40.299999,0.035357,-0.764102,-0.457595
3,1928,Government fixed assets,Current Dollars,9.0,40.700001,0.009877,-0.764019,-0.861384
4,1929,Government fixed assets,Current Dollars,9.0,42.700001,0.047971,-0.763604,-0.257706
...,...,...,...,...,...,...,...,...
92,2017,Government fixed assets,Current Dollars,9.0,14375.299805,0.039641,2.210230,-0.389714
93,2018,Government fixed assets,Current Dollars,9.0,15140.099609,0.051835,2.368917,-0.196465
94,2019,Government fixed assets,Current Dollars,9.0,15694.000000,0.035932,2.483844,-0.448491
95,2020,Government fixed assets,Current Dollars,9.0,16313.700195,0.038727,2.612424,-0.404197


In [ ]:
# Valores específicos que deseas filtrar
specific_table_id = 'FAAt101'
specific_series_code = 'k1wtotl1es00'

df=show_bea_data(engine,"fixed_assets")

# Filtrar el DataFrame basado en esos valores
filtered_df = df[(df['table_id'] == specific_table_id) & (df['series_code'] == specific_series_code)]

# Mostrar el DataFrame filtrado
# filtered_df


duplicated_years = filtered_df[filtered_df['year'].duplicated()]
print(duplicated_years)


Empty DataFrame
Columns: [table_id, series_code, line, line_description, year, metric, units, e, value]
Index: []


In [ ]:
print((df_fix_as['value']<0).sum())

In [ ]:
finagg.bea.api.fixed_assets.get_parameter_list()

In [ ]:
finagg.bea.api.fixed_assets.get_parameter_values("TableName")

In [ ]:
with engine.connect() as connection:
    query = text("SELECT * FROM fixed_assets WHERE table_id = 'FAAt102'")
    df=pd.read_sql(query,connection)
df['units'].unique()


####GDP

In [ ]:
df_gdp=show_bea_data(engine,"gdp_by_industry")
df_gdp

In [ ]:
finagg.bea.api.gdp_by_industry.get_parameter_list()

In [ ]:
finagg.bea.api.gdp_by_industry.get_parameter_values("Industry") #Params: "Frequency", "Industry", "TableID","Year"

In [ ]:
# df_gdp[(df_gdp["table_id"]==1) & (df_gdp["industry"]=="11")]
len(df_gdp["table_id"].unique())

In [ ]:
len(df_gdp["industry"].unique())

In [ ]:
finagg.bea.api.gdp_by_industry.get_parameter_values("industry")

####Transformar datos GDP

In [ ]:
# from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, Index, PrimaryKeyConstraint

# #vamos a generar tantas tablas por cada table_id e infustria
# df_s= df_gdp.sort_values(by=['table_id', 'industry', 'year','freq']) #ordenamos en base a esas columnas
# grouped = df_s.groupby(['table_id', 'industry'])  # generamos df por cada table_id e industry

# metadata = MetaData()

# # Iterar sobre el objeto 'grouped' para definir y crear cada tabla.
# for (table_id, industry), group_df in grouped:
#     # Generar el nombre de la tabla usando el formato especificado.
#     table_name = f"gdp_{table_id}_{industry}"

#     # Excluir las columnas 'table_id' y 'industry'
#     group_df = group_df.drop(columns=['table_id', 'industry', 'freq', 'industry_description'])


#     # Definir la estructura de la tabla
#     table = Table(table_name, metadata,
#               Column('year', Integer),  # Definimos 'year', pero sin marcarlo como clave primaria aquí
#               Column('quarter', Integer),  # Lo mismo con 'quarter'
#               Column('value', Float),
#               PrimaryKeyConstraint('year', 'quarter')  # Definir clave primaria compuesta (no puede hacer mismo año y trimestre)
#               )

#     # Crear la tabla en la base de datos
#     table.create(engine)

#     # Insertar datos en la tabla recién creada
#     group_df.to_sql(table_name, engine, if_exists='append', index=False)


In [ ]:
#Obtener nombre de las tablas generadas

df_s= df_gdp.sort_values(by=['table_id', 'industry', 'year','freq']) #ordenamos en base a esas columnas
grouped = df_s.groupby(['table_id', 'industry'])  # generamos df por cada table_id e industry
generated_tables = []

for (table_id, industry), _ in grouped:
    # Generar el nombre de la tabla usando el formato especificado.
    table_name = f"gdp_{table_id}_{industry}"
    generated_tables.append(table_name)

len(grouped)
# len(df_s["table_id"].unique())*len(df_s["industry"].unique())


####Variables faltantes

In [ ]:
# Obtener valores únicos de table_id e industry
table_ids = df_s['table_id'].unique()
industries = df_s['industry'].unique()

# Crear DataFrames a partir de estos valores únicos
df_table_ids = pd.DataFrame(table_ids, columns=['table_id'])
df_industries = pd.DataFrame(industries, columns=['industry'])

# Agregar una columna clave constante a ambos DataFrames para el cross join
df_table_ids['key'] = 1
df_industries['key'] = 1

# Realizar el cross join
all_combinations = pd.merge(df_table_ids, df_industries, on='key').drop('key', 1)

# Crear una columna clave en ambos DataFrames para identificar combinaciones únicas
df_s['combo_key'] = df_s['table_id'].astype(str) + "_" + df_s['industry']
all_combinations['combo_key'] = all_combinations['table_id'].astype(str) + "_" + all_combinations['industry']

# Identificar las combinaciones faltantes
missing_combinations = all_combinations.loc[~all_combinations['combo_key'].isin(df_s['combo_key'])]

print(missing_combinations)


####Eliminar tablas con variables faltantes

In [ ]:
# from sqlalchemy import create_engine, MetaData, Table

# # Tu conexión previa al engine
# db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
# engine = create_engine(f'sqlite:///{db_path}')

# metadata = MetaData()

# # Lista de industrias para eliminar
# industries_to_drop = ["GDP", "II", "NABI"]

# # Recopila todas las tablas del database
# metadata.reflect(engine)

# for table_name in metadata.tables:
#     for industry in industries_to_drop:
#         if f"_{industry}" in table_name:
#             # Borrar la tabla
#             table_to_drop = Table(table_name, metadata)
#             table_to_drop.drop(engine)
#             print(f"Tabla {table_name} eliminada.")


In [ ]:
#Contar numero de tabalas gdp

from sqlalchemy import create_engine, MetaData

# Tu conexión previa al engine
db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
engine = create_engine(f'sqlite:///{db_path}')

metadata = MetaData()

# Recopila todas las tablas del database
metadata.reflect(engine)

# Filtra las tablas que comienzan con 'gdp_' y cuenta cuántas hay
gdp_tables = [table_name for table_name in metadata.tables if table_name.startswith('gdp_')]
count_gdp_tables = len(gdp_tables)

print(f"El número de tablas que comienzan con 'gdp_' es: {count_gdp_tables}")


####Chequear si tienen valores NaN

In [ ]:
from sqlalchemy import text

def count_tables_with_nan(engine, table_prefix):
    # Usando el reflejo para obtener todas las tablas en la base de datos
    metadata = MetaData()
    metadata.reflect(engine)

    # Filtra las tablas que comienzan con el prefijo dado
    tables = [table_name for table_name in metadata.tables if table_name.startswith(table_prefix)]

    # Cuenta las tablas que tienen NaN
    count_nan_tables = 0

    for table_name in tables:
        df = show_bea_data(engine, table_name)

        # Si el DataFrame tiene NaN, incrementa el contador
        if df.isnull().values.any():
            count_nan_tables += 1

    print(f"De las {len(tables)} tablas que comienzan con '{table_prefix}', {count_nan_tables} tienen valores NaN.")

# Usando la función
count_tables_with_nan(engine, 'gdp_')



####Chequeo general

In [ ]:
from sqlalchemy import MetaData, text

def verify_data_quality(engine, table_prefix):
    # Usando el reflejo para obtener todas las tablas en la base de datos
    metadata = MetaData()
    metadata.reflect(engine)

    # Filtra las tablas que comienzan con el prefijo dado
    tables = [table_name for table_name in metadata.tables if table_name.startswith(table_prefix)]

    for table_name in tables:
        print(f"Verificando la tabla {table_name}...")
        df = show_bea_data(engine, table_name)

        # Chequeo de valores duplicados
        duplicated_rows = df.duplicated().sum()
        if duplicated_rows > 0:
            print(f"- Hay {duplicated_rows} filas duplicadas.")

        # Chequeo de valores extremos en columnas numéricas
        for col in df.select_dtypes(['float64', 'int64']):
            quantile_1 = df[col].quantile(0.25)
            quantile_3 = df[col].quantile(0.75)
            iqr = quantile_3 - quantile_1
            lower_bound = quantile_1 - (1.5 * iqr)
            upper_bound = quantile_3 + (1.5 * iqr)
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
            if not outliers.empty:
                print(f"- La columna {col} tiene {len(outliers)} valores atípicos.")

        # Chequeo de tipos de datos y valores únicos en columnas categóricas
        for col in df.select_dtypes(['object']):
            unique_values = df[col].nunique()
            if unique_values < len(df) * 0.05:  # Suponiendo que si hay menos del 5% de valores únicos, podría ser categórico
                print(f"- La columna {col} tiene {unique_values} valores únicos.")

        # Chequeo de valores cero
        zero_values_cols = df.columns[(df == 0).any()]
        for col in zero_values_cols:
            zero_count = (df[col] == 0).sum()
            print(f"- La columna {col} tiene {zero_count} valores cero.")

        print("\n")

# Usando la función
verify_data_quality(engine, 'gdp_')


In [ ]:
df_ex=show_bea_data(engine,"gdp_209_721")
df_ex

####Aplicar Log y normalizacion

In [ ]:
# #Aplicar Log
# df_ex["log_change"]=finagg.utils.safe_log_change(df_ex["value"])
# finagg.utils.safe_pct_change
# #Aplicar Z-score
# df_ex['norm_log_change'] = (df_ex['log_change'] - df_ex['log_change'].mean()) / df_ex['log_change'].std()

# df_ex.dropna(subset=['log_change'], inplace=True)  #Borramos filas NaN

# df_ex

####Generar tablas norm z-score

In [ ]:
# from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, Index, PrimaryKeyConstraint
# import finagg

# db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
# engine = create_engine(f'sqlite:///{db_path}')

# df_gdp=show_bea_data(engine,"gdp_by_industry")
# df_gdp


# # Ordenar y agrupar el DataFrame
# df_s = df_gdp.sort_values(by=['table_id', 'industry', 'year', 'freq'])
# grouped = df_s.groupby(['table_id', 'industry'])

# metadata = MetaData()

# # Iterar sobre el objeto 'grouped'
# for (table_id, industry), group_df in grouped:
#     table_name = f"gdp_{table_id}_{industry}_z_norm"

#     # Excluir las columnas que no queremos
#     group_df = group_df.drop(columns=['table_id', 'industry', 'freq', 'industry_description'])

#     # Calcular log_change y norm_log_change
#     group_df['norm'] = (group_df['value'] - group_df['value'].mean()) / group_df['value'].std()  #Normalizacion z-score

#     # Definir la estructura de la tabla
#     table = Table(table_name, metadata,
#                   Column('year', Integer),
#                   Column('quarter', Integer),
#                   Column('value', Float),
#                   Column('norm', Float),   # Nueva columna
#                   PrimaryKeyConstraint('year', 'quarter')
#                  )

#     # Crear la tabla en la base de datos
#     table.create(engine)

#     # Insertar datos en la tabla recién creada
#     group_df.to_sql(table_name, engine, if_exists='append', index=False)


####Eliminar tablas de industrias que faltan datos

In [ ]:
from sqlalchemy import create_engine, MetaData, Table

# Tu conexión previa al engine
db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
engine = create_engine(f'sqlite:///{db_path}')

metadata = MetaData()

# Lista de industrias para eliminar
industries_to_drop = ["GDP", "II", "NABI"]

# Recopila todas las tablas del database
metadata.reflect(engine)

for table_name in metadata.tables:
    for industry in industries_to_drop:
        if f"_{industry}" in table_name:
            # Borrar la tabla
            table_to_drop = Table(table_name, metadata)
            table_to_drop.drop(engine)
            print(f"Tabla {table_name} eliminada.")

In [ ]:
df_ex=show_bea_data(engine,"gdp_9_GFG_z_norm")
df_ex

####Eliminar primer valor NaN

In [ ]:
# Obtener todos los nombres de las tablas en la base de datos
with engine.connect() as conn:
    result = conn.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))
    all_tables = [row[0] for row in result]

# Filtrar solo las tablas que coinciden con el patrón "gdp_..._log_norm"
relevant_tables = [table_name for table_name in all_tables if table_name.startswith("gdp_") and table_name.endswith("_log_norm")]
print(relevant_tables)

# Limpiar cada tabla
for table_name in relevant_tables:
    with engine.connect() as conn:
        query = f"DELETE FROM {table_name} WHERE log_change IS NULL OR norm_log_change IS NULL"
        conn.execute(text(query))
        conn.commit()


In [ ]:
table_name = "gdp_14_GFGN_log_norm"  # Usar el nombre de una tabla como ejemplo
with engine.connect() as conn:
    result = conn.execute(text(f"SELECT COUNT(*) FROM {table_name} WHERE log_change IS NULL OR norm_log_change IS NULL"))
    count = result.scalar()
print(count)


####GDP FRED

In [ ]:
db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
engine = create_engine(f'sqlite:///{db_path}')

table = '"fred.refined.economic"'
with engine.begin() as conn:
    query = text(f"""SELECT * FROM {table}""")
    df = pd.read_sql_query(query, conn)

# len(df["LOG_CHANGE(GDP)"].unique())
df

###Pruebas

In [ ]:
finagg.sec.feat.quarterly.normalized.from_other_refined

In [ ]:
###Borrar columnas: Index